In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Convolution2D, Activation, Flatten, MaxPooling2D,Input,Dropout,GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.framework import ops
from binarization_utils import *

In [ ]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()  

x_train = x_train/255.0  
x_test = x_test/255.0  

In [ ]:
import os
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
# x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
# x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x_train[0].reshape(28,28), cmap="gray")

In [ ]:
resid_levels = True
BINARY = False
LOGIC_SHRINKAGE = False
custom_rand_seed = 1

In [ ]:
autoencoder = Sequential()

# Encoder
autoencoder.add(binary_dense(levels=resid_levels, n_in=784, n_out=128, input_shape=(784,), BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="binary_dense_encoder_1"))
autoencoder.add(Activation('relu'))

autoencoder.add(binary_dense(levels=resid_levels, n_in=128, n_out=64, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="binary_dense_encoder_2"))
autoencoder.add(Activation('relu'))

autoencoder.add(binary_dense(levels=resid_levels, n_in=64, n_out=9, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="binary_dense_encoder_3"))
autoencoder.add(Activation('relu'))

# Decoder
autoencoder.add(binary_dense(levels=resid_levels, n_in=9, n_out=64, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="binary_dense_decoder_1"))
autoencoder.add(Activation('relu'))

autoencoder.add(binary_dense(levels=resid_levels, n_in=64, n_out=128, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="binary_dense_decoder_2"))
autoencoder.add(Activation('relu'))

autoencoder.add(binary_dense(levels=resid_levels, n_in=128, n_out=784, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="binary_dense_decoder_3"))
autoencoder.add(Activation('sigmoid'))

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
epochs=3

for epoch in range(epochs):

    history = autoencoder.fit(
      x_train,
      x_train,
      epochs=1, 
      batch_size=32, validation_split=0.10
        )   

In [ ]:
plt.imshow(x_test[0].reshape(28,28), cmap="gray")

In [ ]:
ae_out = autoencoder.predict([ x_test[0].reshape(1, 28* 28) ])
img = ae_out[0]  # predict is done on a vector
plt.imshow(ae_out[0].reshape(28,28), cmap="gray")